
Downloading the preprocessed .csv file of images from google drive

In [ ]:
! pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.13.0-py3-none-any.whl size=9034 sha256=3d837f1400ca0766401874a8db470cab4270ef82ddc51093b7a00503ae716034
  Stored in directory: /root/.cache/pip/wheels/2f/2a/2f/86449b6bdbaa9aef873f68332b68be6bfbc386b9219f47157d
Successfully built gdown


In [ ]:
import gdown; url = 'https://drive.google.com/uc?export=download&id=1lF-Y3XE-Cp2LacG8HDXLU8OWYE84x9dw' 
output = 'img.csv' 
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?export=download&id=1lF-Y3XE-Cp2LacG8HDXLU8OWYE84x9dw
To: /kaggle/working/img.csv
2.63GB [00:13, 192MB/s] 


'img.csv'

Importing the required modules

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from tqdm import tqdm
import tensorflow as tf
%matplotlib inline


Reading the .csv into dataframe and normlizing it by dividing it by 255

In [ ]:
df=pd.read_csv('img.csv')

In [ ]:
df=df/255

storing the images as numpy array reshaping it

In [ ]:
X=np.asarray(df)
X=X.reshape(18632,100,100,3)


Reading the training .csv

In [ ]:
df_train=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

**Splitting the labels into 6 classes and encoding it**

In [ ]:
df_train["labels"]=df_train["labels"].apply(lambda x:x.split(" "))

In [ ]:
df_train['healthy']=0
df_train['scab']=0
df_train['complex']=0
df_train['rust']=0
df_train['frog_eye_leaf_spot']=0
df_train['powdery_mildew']=0


In [ ]:
for rown in range(0,len(df_train['labels'])):
    for i in df_train['labels'][rown]:
        if i == 'healthy':
            df_train['healthy'][rown]=1
        elif i == 'scab':
            df_train['scab'][rown]=1
        elif i == 'complex':
            df_train['complex'][rown]=1
        elif i == 'rust':
            df_train['rust'][rown]=1
        elif i == 'frog_eye_leaf_spot':
            df_train['frog_eye_leaf_spot'][rown]=1
        elif i == 'powdery_mildew':
            df_train['powdery_mildew'][rown]=1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykerne

In [ ]:
y=np.asarray(df_train[['healthy','scab','complex','rust','frog_eye_leaf_spot','powdery_mildew']])

In [ ]:
y

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2,random_state=42)


Creating the Architecture of CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters=512, kernel_size=(5, 5), activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss="binary_crossentropy",
                metrics=['accuracy'])


Training the model

In [ ]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=50)


Epoch 1/50
299/299 [==============================] - 49s 131ms/step - loss: 0.5893 - accuracy: 0.1876 - val_loss: 0.5105 - val_accuracy: 0.2984
Epoch 2/50
299/299 [==============================] - 34s 113ms/step - loss: 0.4925 - accuracy: 0.2447 - val_loss: 0.4528 - val_accuracy: 0.3094
Epoch 3/50
299/299 [==============================] - 34s 113ms/step - loss: 0.4676 - accuracy: 0.2813 - val_loss: 0.4666 - val_accuracy: 0.3762
Epoch 4/50
299/299 [==============================] - 34s 113ms/step - loss: 0.4560 - accuracy: 0.2999 - val_loss: 0.4313 - val_accuracy: 0.3872
Epoch 5/50
299/299 [==============================] - 34s 113ms/step - loss: 0.4430 - accuracy: 0.3266 - val_loss: 0.4239 - val_accuracy: 0.4060
Epoch 6/50
299/299 [==============================] - 34s 112ms/step - loss: 0.4355 - accuracy: 0.3444 - val_loss: 0.4214 - val_accuracy: 0.4245
Epoch 7/50
299/299 [==============================] - 34s 113ms/step - loss: 0.4276 - accuracy: 0.3582 - val_loss: 0.4030 - val_ac